# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
vacation_df = pd.read_csv("output_data/weather.csv")
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,24.12,60,100,3.00,US,1643389936
1,leningradskiy,69.3833,178.4167,-11.61,96,97,1.97,RU,1643389941
2,port augusta,-32.5000,137.7667,70.97,69,64,16.69,AU,1643389941
3,beringovskiy,63.0500,179.3167,-6.88,94,83,3.13,RU,1643389942
4,seeboden,46.8219,13.4950,39.25,90,17,5.66,AT,1643389942


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store lat and lng in locations
locations = vacation_df[["Lat","Lng"]]
weight = vacation_df["Humidity"].astype(float)

#drop NaN values
vacation_df = vacation_df.dropna()

# Heat Map
fig = gmaps.figure(map_type="HYBRID")

# Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=10, point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
conditions = vacation_df.loc[(vacation_df["Max Temp"] > 72) & (vacation_df["Max Temp"] <= 80) & (vacation_df["Humidity"] < 50)]
conditions

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,nandigama,16.7833,80.3000,73.20,42,83,0.51,IN,1643389997
182,tessalit,20.1986,1.0114,77.99,9,5,9.95,ML,1643390022
342,necochea,-38.5473,-58.7368,77.90,38,0,1.50,AR,1643390091
365,arlit,18.7369,7.3853,74.59,12,0,5.77,NE,1643390102
380,nizwa,22.9333,57.5333,72.41,32,0,2.35,OM,1643390108
391,guamuchil,25.4667,-108.1000,74.37,33,19,2.77,MX,1643389962
396,vredendal,-31.6683,18.5012,74.70,48,0,26.84,ZA,1643390115
419,san cristobal,7.7669,-72.2250,77.18,40,77,2.17,VE,1643390126
435,birao,10.2849,22.7882,75.81,15,72,6.17,CF,1643390133
471,wau,7.7029,27.9953,78.26,19,83,5.53,SS,1643390150


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = conditions
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nandigama,16.7833,80.3000,73.20,42,83,0.51,IN,1643389997
1,tessalit,20.1986,1.0114,77.99,9,5,9.95,ML,1643390022
2,necochea,-38.5473,-58.7368,77.90,38,0,1.50,AR,1643390091
3,arlit,18.7369,7.3853,74.59,12,0,5.77,NE,1643390102
4,nizwa,22.9333,57.5333,72.41,32,0,2.35,OM,1643390108
5,guamuchil,25.4667,-108.1000,74.37,33,19,2.77,MX,1643389962
6,vredendal,-31.6683,18.5012,74.70,48,0,26.84,ZA,1643390115
7,san cristobal,7.7669,-72.2250,77.18,40,77,2.17,VE,1643390126
8,birao,10.2849,22.7882,75.81,15,72,6.17,CF,1643390133
9,wau,7.7029,27.9953,78.26,19,83,5.53,SS,1643390150


In [51]:
hotel_df["Hotel Name"] = ""

#parameters
params={
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key":g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}:{row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
    
    print("----------------")



<ipython-input-51-1f3e1be90b32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Retrieving Results for Index 0:nandigama.
Closest hotel is Hotel VGS inn.
----------------
Retrieving Results for Index 1:tessalit.


C:\Users\foh01\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result...skipping.
----------------
Retrieving Results for Index 2:necochea.
Closest hotel is Hotel Moderno.
----------------
Retrieving Results for Index 3:arlit.
Closest hotel is Hôtel Temet.
----------------
Retrieving Results for Index 4:nizwa.
Closest hotel is Nizwa Hotel Apartments.
----------------
Retrieving Results for Index 5:guamuchil.
Closest hotel is Hotel Flores.
----------------
Retrieving Results for Index 6:vredendal.
Closest hotel is River Lodge.
----------------
Retrieving Results for Index 7:san cristobal.
Closest hotel is Lidotel Hotel Boutique San Cristobal.
----------------
Retrieving Results for Index 8:birao.
Missing field/result...skipping.
----------------
Retrieving Results for Index 9:wau.
Closest hotel is Royal castle Hotel wau, South Sudan.
----------------
Retrieving Results for Index 10:young.
Closest hotel is Hotel Martinez.
----------------
Retrieving Results for Index 11:rio grande.
Closest hotel is Hotel Atlântico Rio Grande.
---------

In [52]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,nandigama,16.7833,80.3000,73.20,42,83,0.51,IN,1643389997,Hotel VGS inn
1,tessalit,20.1986,1.0114,77.99,9,5,9.95,ML,1643390022,
2,necochea,-38.5473,-58.7368,77.90,38,0,1.50,AR,1643390091,Hotel Moderno
3,arlit,18.7369,7.3853,74.59,12,0,5.77,NE,1643390102,Hôtel Temet
4,nizwa,22.9333,57.5333,72.41,32,0,2.35,OM,1643390108,Nizwa Hotel Apartments
5,guamuchil,25.4667,-108.1000,74.37,33,19,2.77,MX,1643389962,Hotel Flores
6,vredendal,-31.6683,18.5012,74.70,48,0,26.84,ZA,1643390115,River Lodge
7,san cristobal,7.7669,-72.2250,77.18,40,77,2.17,VE,1643390126,Lidotel Hotel Boutique San Cristobal
8,birao,10.2849,22.7882,75.81,15,72,6.17,CF,1643390133,
9,wau,7.7029,27.9953,78.26,19,83,5.53,SS,1643390150,"Royal castle Hotel wau, South Sudan"


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [60]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

figure_layout={
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
}

infobox =gmaps.symbol_layer(
    locations,
    info_box_content=hotel_info
)

fig = gmaps.figure(layout=figure_layout)

# Display figure
fig.add_layer(markers)
fig.add_layer(infobox)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='800px'))